In [ ]:
#D52_Qudit_heralded_Ramsey.py created 2024-09-26 15:04:20.886427

import matlab.engine
eng = matlab.engine.start_matlab()
eng.addpath(r'C:\Users\ions\Documents\MATLAB\iqtools',nargout = 0)
eng.SendAWGCommand("RES", nargout=0)
time.sleep(1)

import sys
import os
import datetime
import glob
import json
import numpy as np
import shutil
from scipy.optimize import curve_fit
sys.path.append(r'C:\Users\ions\Documents\IonControl\project_dir\QuditLab\config\Scripts')
from Functions_Data import *
from Functions_AWG import *
from Functions_Calibration import *
from Functions_Measurement import *
from Functions_Gates import *
script_functions = (getGlobal, setGlobal, setScan, startScan, stopScan, getAllData, createTrace, closeTrace, plotPoint, scriptIsStopped, setEvaluation)
dt_string = datetime.datetime.now().strftime("%Y%m%d_%H%M")
year = datetime.datetime.now().strftime("%Y")
month = datetime.datetime.now().strftime("%m")
day = datetime.datetime.now().strftime("%d")
pattern = "Z:\Lab Data\Qudit_Ramsey_raw_data\Raw_data\qudit_ramsey_scan_*"
def insert(x,p):
    return x[:int((len(x))/2)] + [p] +x[int((len(x))/2):]
Side_band_cooling_reps = 0


#initial_state = [[0,2,2]]

do_calibrations_0 = True
Measure_U1_only = False

#ramsey_wait_times = np.arange(0,5100,500)
repeat_experiments = 1
hidden_bits = ["00", "01", "10", "11"]
dim = 5
sub_dim = 4
mappings = {0: '-2', 1: '[2, -1]', 2: '[3, -2]', 3: '[4, -4]', 4: '[3, -3]'}
for hidden_string in hidden_bits:

    f_offset = float(getGlobal('f_offset'))
    f_upper = float(getGlobal('f_upper'))

    pitime_n2 = float(getGlobal('pitime_n2')) # [-2, 4, -4]
    pitime_n1 = float(getGlobal('pitime_n1')) # [-2, 3, -3]
    pitime_0 = float(getGlobal('pitime_0')) # [2, 4, 2]
    pitime_p1 = float(getGlobal('pitime_p1')) # [2, 4, 3]
    pitime_p2 = float(getGlobal('pitime_p2')) # [2, 4, 4]


    dimension = int(np.max(simulated_phase_mask_U2))+1
    dimension_string = str(dimension)

    list_of_inits = [[[-1,3,-2],[0,4,-2],[1,4,0],[2,4,2]],
    [[-2,3,-1],[0,3,0],[1,4,0],[2,4,3]],
    [[-2,2,-1],[-1,3,0],[1,3,2],[2,4,2]],
    [[-2,3,-1],[-1,3,0],[0,3,2],[2,4,3]],
    [[-2,2,-1],[-1,3,0],[0,3,2],[1,3,3]]
    ]

    def Z_gate_states(sub_dim, hidden_string):
    
        n = int(np.log2(sub_dim))
        if len(hidden_string) != n:
            raise ValueError("Hidden string length must equal log2(sub_dim).")
        s_bits = [int(bit) for bit in hidden_string]
        list_of_bits = []
        for x in range(sub_dim):
            x_bits = [int(b) for b in format(x, f'0{n}b')]
            dot = sum(s * xb for s, xb in zip(s_bits, x_bits)) % 2
            if dot == 1:
                list_of_bits.append(x+1)
        return list_of_bits

    H1_pulses, _, H1_phases, H1_angles, probe_trans = Bussed_PolyQubit_Hadamard(dim, mappings,pi_t = [pitime_n2,pitime_n1,pitime_0,pitime_p1,pitime_p2],Virtual_Z = False, Z_gate_state = None) 
    H2_pulses, _, H2_phases, H2_angles, _ = Bussed_PolyQubit_Hadamard(dim, mappings,pi_t = [pitime_n2,pitime_n1,pitime_0,pitime_p1,pitime_p2],Virtual_Z = True, Z_gate_state = Z_gate_states(sub_dim, hidden_string))
    pulse_train = H1_pulses + H2_pulses
    phase_shifts = H1_phases + H2_phases
    angles = H1_angles + H2_angles


    s12_level = H1_pulses[0][0]

    F1PumpTime = 1.5 #us
    F1PumpReps = 60
    InitReps = 0
    fs = 2e9
    threshold = 11

    pulse_program = "PolyQubit_BVA_pulse_program"
            
    def findPiTime_withfit_plotPD(threshold ,pulse_program, script_functions):
        getGlobal, setGlobal, setScan, startScan, stopScan, getAllData, createTrace, closeTrace, plotPoint, scriptIsStopped, setEvaluation = script_functions

        BVA_dummy = 0
        file_names_list_all = []
        file_names_list = []
        while BVA_dummy<1:
            BVA_dummy += BVA_dummy

            if scriptIsStopped():
                eng.quit()
                break
    
            print('1')
            setEvaluation('Eval2')
            if pulse_time == 0:
                if do_calibrations:
                    delta = 0
                    ramsey_real_wait_time = 150
                    freq_offset = run_fast_calibration(script_functions,delta,ramsey_real_wait_time,target_transition=[0,2,0],f_offset_input=None,f_upper_input=None,ref_pitimes=None)
                    freq_upper = run_fast_calibration(script_functions,delta,ramsey_real_wait_time,target_transition=[-1,4,-3],f_offset_input=None,f_upper_input=None,ref_pitimes=None)
            setEvaluation('Eval3')
    
            f_offset = float(getGlobal('f_offset'))
            f_upper = float(getGlobal('f_upper'))
    
            pitime_n2 = float(getGlobal('pitime_n2')) # [-2, 4, -4]
            pitime_n1 = float(getGlobal('pitime_n1')) # [-2, 3, -3]
            pitime_0 = float(getGlobal('pitime_0')) # [2, 4, 2]
            pitime_p1 = float(getGlobal('pitime_p1')) # [2, 4, 3]
            pitime_p2 = float(getGlobal('pitime_p2')) # [2, 4, 4]
    
    
            init_trans = list_of_inits[s12_level+2]
            init_freqs_array = list(Get_1762_EOM_Freqs_an1an2(init_trans,f_offset,f_upper))
            init_times_array = list(Get_1762_PiTimes(init_trans,pitime_n2,pitime_n1,pitime_0,pitime_p1,pitime_p2))
            init_pulse_time = sum(init_times_array) 
    
            probe_freq_list = list(Get_1762_EOM_Freqs_an1an2(probe_trans,f_offset,f_upper))
    
            initial_state_frequency = list(Get_1762_EOM_Freqs_an1an2(initial_state,f_offset,f_upper))
    
            pulse_train_frequencies = list(Get_1762_EOM_Freqs_an1an2(pulse_train,f_offset,f_upper))
    
            pi_time_initial_state = list(Get_1762_PiTimes(initial_state,pitime_n2,pitime_n1,pitime_0,pitime_p1,pitime_p2))
    
            pi_time_pulse_train = list(Get_1762_PiTimes(pulse_train,pitime_n2,pitime_n1,pitime_0,pitime_p1,pitime_p2))
    
            pi_times_probe_trans = list(Get_1762_PiTimes(probe_trans,pitime_n2,pitime_n1,pitime_0,pitime_p1,pitime_p2))
    
            print(pulse_train)
    
            F1_Pump_Time = getGlobal('F1_PumpTime')
            if not "%s"%F1_Pump_Time == "%s us"%F1PumpTime:
                setGlobal("F1_PumpTime", F1PumpTime, "us")
    
            Init_reps = getGlobal('InitialisationReps')
            if not "%s"%Init_reps == InitReps:
                setGlobal("InitialisationReps", InitReps, "")
    
            F1Pump_reps = getGlobal('F1_PumpReps')
            if not "%s"%F1Pump_reps == F1PumpReps:
                setGlobal("F1_PumpReps", F1PumpReps, "")
    
            Init_PulseTime = getGlobal('Init_Shelving_Pulse_Time')
            if not "%s"%Init_PulseTime == init_pulse_time:
                setGlobal("Init_Shelving_Pulse_Time", init_pulse_time, "us")
    
            SB_cooling_reps = getGlobal("Sideband_Cooling_Reps")
            if not "%s"%SB_cooling_reps == Side_band_cooling_reps:
                setGlobal("Sideband_Cooling_Reps", Side_band_cooling_reps, "")
    
            corrected_pulse_train_times = []

            for i,angle in enumerate(angles):
                corrected_pulse_time = pi_time_pulse_train[i]*angle
                corrected_pulse_train_times.append(corrected_pulse_time)
            print(corrected_pulse_train_times)

            AOM_ON_time = getGlobal('Shelving_Pulse_Time')
            if not "%s"%AOM_ON_time == "%s us"%sum(corrected_pulse_train_times):
                setGlobal("Shelving_Pulse_Time", sum(corrected_pulse_train_times), "us")
            print('2')
    
    # Phase Calculations ===================================================================

            full_phases = list(np.pi*np.array(phase_shifts))
            print(full_phases)
    #=======================================================================================
    # AWG commands======================================================================================
            eng.SendAWGCommand("RES", nargout=0)
            time.sleep(5)
            eng.SendAWGCommand("TRAC:DEL:ALL",nargout = 0)
            eng.SendAWGCommand("SOUR:SEQ:DEL:ALL",nargout = 0)
            eng.SendAWGCommand("INIT:CONT 0", nargout=0) #0 sets AWG to externally triggered mode
            eng.SendAWGCommand("OUTP:STAT 1", nargout=0)
            eng.SendAWGCommand("SOUR:FUNC:MODE USER",nargout = 0)
    
            eng.SendAWGCommand("ROSC:SOUR EXT", nargout=0)
            time.sleep(0.1)
            eng.SendAWGCommand("ROSC:SOUR INT", nargout=0)
    
            eng.SendAWGCommand("TRAC:DEL:ALL",nargout = 0)
            eng.SendAWGCommand("SOUR:SEQ:DEL:ALL",nargout = 0)
    
            print('3')
    #---------------------------------------------------------------------------- S1/2 inits
            matlab_init_freqs = matlab.double(init_freqs_array)
            matlab_init_times = matlab.double(init_times_array)
    #---------------------------------------------------------------------------- D5/2 init pulse

            print(initial_state_frequency)
            print(pi_time_initial_state)
            consolePrint(str(initial_state_frequency))
            consolePrint(str(pi_time_initial_state))
            
            matlab_initial_state_freq = matlab.double(initial_state_frequency)
            matlab_initial_state_pulse_time = matlab.double(pi_time_initial_state)
            matlab_initial_state_phases = matlab.double([0])
    #---------------------------------------------------------------------------------------------------------- pulse_train
            print(pulse_train_frequencies)
            print(corrected_pulse_train_times)
            matlab_pulse_train_freqs = matlab.double(pulse_train_frequencies)
            matlab_pulse_train_times = matlab.double(corrected_pulse_train_times)
            matlab_pulse_train_phases = matlab.double(full_phases)
    
            print(len(pulse_train_frequencies),len(corrected_pulse_train_times),len(full_phases))
    
            #for i in range(15):
            #    print(pulse_train_frequencies[i],corrected_pulse_train_times[i])
    
    #---------------------------------------------------------------------------- Uploading
            power_factor = matlab.double([1])
            power_factor_dbm = matlab.double([1])
            print('4.5')
            seg_num = 1
            eng.Pulse_upload(matlab_init_freqs,matlab_init_times,fs,seg_num,power_factor,nargout = 0)
            if scriptIsStopped():
                eng.quit()
                break
            
            seg_num = seg_num + 1 
            eng.Pulse_upload_dummy(fs,seg_num,nargout = 0) 
            print('5')
            seg_num = seg_num + 1
            eng.Pulse_upload_with_phases(matlab_initial_state_freq,matlab_initial_state_pulse_time,matlab_initial_state_phases,fs,seg_num,nargout = 0)
            
            seg_num = seg_num + 1 
            eng.Pulse_upload_dummy(fs,seg_num,nargout = 0) 
    #++++++++++++++++++++++++++Herald will be here++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
            seg_num = seg_num + 1
            eng.Pulse_upload_with_phases(matlab_pulse_train_freqs,matlab_pulse_train_times,matlab_pulse_train_phases,fs,seg_num,nargout = 0)
            if scriptIsStopped():
                eng.quit()
                break
            seg_num = seg_num + 1
            eng.Pulse_upload_dummy(fs,seg_num,nargout = 0) # Dummy 3rd sequence
            print('6')
            # Readout pulses
            for idx, freq in enumerate(probe_freq_list):
                matlab_ro_freq = matlab.double([freq])
                matlab_ro_time = matlab.double([pi_times_probe_trans[idx]])
                print(seg_num,[freq],[pi_times_probe_trans[idx]])
                seg_num = seg_num + 1
                eng.Pulse_upload(matlab_ro_freq,matlab_ro_time,fs,seg_num,power_factor,nargout = 0)
                if scriptIsStopped():
                    eng.quit()
                    break
                seg_num = seg_num + 1
                eng.Pulse_upload_dummy(fs,seg_num,nargout = 0) # Dummy 3rd sequence
    
    #-------------------------------------------------------------------------------------------------------------------------
            print('7')
    
            eng.SendAWGCommand("SOUR:SEQ:DEF 1,1,1,1", nargout=0)
            eng.SendAWGCommand("SOUR:SEQ:DEF 2,2,1,1", nargout=0)
            eng.SendAWGCommand("SOUR:SEQ:DEF 3,3,1,0", nargout=0)
            eng.SendAWGCommand("SOUR:SEQ:DEF 4,4,1,1", nargout=0)
            eng.SendAWGCommand("SOUR:SEQ:DEF 5,5,1,0", nargout=0)
            eng.SendAWGCommand("SOUR:SEQ:DEF 6,6,1,1", nargout=0)
            seq_num = 6
    
            for idx, freq in enumerate(probe_freq_list):
                seq_num = seq_num + 1
                string = f"SOUR:SEQ:DEF {seq_num}, {seq_num},1,0"
                eng.SendAWGCommand(string, nargout=0)
                seq_num = seq_num + 1
                string = f"SOUR:SEQ:DEF {seq_num}, {seq_num},1,1"
                eng.SendAWGCommand(string, nargout=0)
    
            Table_length_dummy =  getGlobal('Table_length')
            if not "%s"%Table_length_dummy == seq_num:
                setGlobal("Table_length", seq_num, "")
    
            eng.SendAWGCommand("SOUR:FUNC:MODE SEQ",nargout = 0)
    
            eng.SendAWGCommand("OUTP:COUP DC",nargout = 0)
    
            eng.SendAWGCommand("SOUR:VOLT:LEV:AMPL 1.5",nargout = 0)
    
            setScan(pulse_program)
            startScan(globalOverrides=list(), wait=True)
            stopScan()
    
            data = getAllData()
            herald_data = data['PMT Index 0'][1]
            print(herald_data)
            ket1_data = data['PMT Index 1'][1]
            print(ket1_data)
            ket2_data = data['PMT Index 2'][1]
            print(ket2_data)
            ket3_data = data['PMT Index 3'][1]
            print(ket3_data)
            ket4_data = data['PMT Index 4'][1]
            print(ket4_data)
    
            arrays = []
            for idx,herald_outcome in enumerate(herald_data):
                if herald_outcome < threshold:# and ket1_data[idx] < threshold:
                    if ket1_data > threshold or ket2_data > threshold or ket3_data > threshold or ket4_data > threshold: 
                        arrays.append([ket1_data[idx],ket2_data[idx],ket3_data[idx],ket4_data[idx]])
            mean_value = np.mean(np.array(arrays)>threshold , axis = 0)
    
    #--------------------------------------------------------Raw_data_file_saving-----------------------------------------------------------------------------------------------

            year = datetime.datetime.now().strftime("%Y")
            month = datetime.datetime.now().strftime("%m")
            day = datetime.datetime.now().strftime("%d")

            file_path_today = f'C:\\Users\\ions\\Documents\\IonControl\\project_dir\\QuditLab\\{year}\\{year}_{month}\\{year}_{month}_{day}\\'
            destination_today = f'Z:\\Lab Data\\Algorithms\\Polyqubit_BVA\\IonControl_raw_data_copied\\{year}\\{year}_{month}\\{year}_{month}_{day}\\'
            pattern = f'C:\\Users\\ions\\Documents\\IonControl\\project_dir\\QuditLab\\{year}\\{year}_{month}\\{year}_{month}_{day}\\BVA_2_polyqubit_*'
            soruce_file_path = f'C:\\Users\\ions\\Documents\\IonControl\\project_dir\\QuditLab\\{year}\\{year}_{month}\\{year}_{month}_{day}\\BVA_2_polyqubit'

            def copy_file(src_file, dest_path):
             
                if not os.path.isfile(src_file):
                    print(f"Source file does not exist: {src_file}")
                    return
             
                dest_folder = os.path.dirname(dest_path)
             
                if not os.path.exists(dest_folder):
                    os.makedirs(dest_folder)
                    print(f"Created destination folder: {dest_folder}")
             
                shutil.copy2(src_file, dest_path)
                print(f"File copied from {src_file} to {dest_path}")

            #if not file_names_list_all:
            matching_files = glob.glob(pattern)
            matching_files = sorted(matching_files, key=lambda t: os.stat(t).st_mtime)
            file_path = matching_files[-1]
            chunks = file_path.split('\\')
            print(chunks[:-1])        
            fname = chunks[-1]
            file_names_list_all.append(destination_today+fname)
            file_names_list.append(destination_today+fname)
            source_file = file_path_today + fname
            copied_file = destination_today + fname
            copy_file(source_file,copied_file)
            
            
            if np.isnan(mean_value):
                mean_value = 1
            PD = mean_value
            print("PD is", PD)

            filename = f'Z:\\Lab Data\\Algorithms\\Polyqubit_BVA\\BVA_logs\\BVA_{int(np.log2(sub_dim))}_polyqubit_hiddenstring_{hidden_string}_{dt_string}.txt'
            with open(filename,'w') as file:

                file.write(f"{PD}\n")
                file.write(f"{pulse_train}\n")
                file.write(f"{[f_offset,f_upper]}\n")
                file.write(f"{pi_time_pulse_train}\n")
                file.write(f"{corrected_pulse_train_times}\n")
                file.write(f"{probe_trans}\n")
                file.write(f"{phase_shifts}\n")




    print('this worked')
    setEvaluation('Eval3')
    pi_time = findPiTime_withfit_plotPD(threshold, pulse_program, script_functions)
    print(pi_time)
    setEvaluation('Eval2')


eng.quit()

In [ ]:
###########################################################################
#
#  pulse a number of times for script - trapping Ba138 and other even isotopes
#


# masks and shutters
shutter ShutterInitialization
masked_shutter ShelvingLaser
masked_shutter INIT_ShelvingLaser
masked_shutter FluorescenceLasers
masked_shutter RepumpShelve
masked_shutter OpticalPumpLasers
masked_shutter ShelvingCoolLaserOne
masked_shutter ShelvingCoolLaserTwo
masked_shutter AWGTrigger
masked_shutter F1PumpLasers
masked_shutter AllLasersOn
masked_shutter Just650Laser
masked_shutter Shutter614nm
masked_shutter F1MechShutter

#masked_shutter test



#counter
counter PMT

# times
parameter PulseTime
parameter InitPulseTime
parameter GeneralWaitTime 
parameter RepumpTime
parameter PMTIntTime
parameter OpticalPumpTime
parameter OpticalPumpTimeForCool
parameter RepumpTimeForCool
parameter CoolTimeStart
parameter CoolTimeFluor
parameter F1PumpTime
parameter F1PumpTime_long
parameter LineTriggerBool
parameter AWGEventTriggerWait
parameter DCrystalWait
parameter MechShutter614Wait
parameter F1MechShutterWait
parameter RamseyWaitTime
# control parameters
parameter DummyVar
parameter DummyAWGTableLen
parameter Experiments
parameter CoolCycleNumOne
parameter CoolCycleNumTwo
parameter CoolPulseStep
parameter InitReps
parameter F1PumpReps
parameter Total_segs

#variables
var ExperimentCount
var Repetitions
var CoolCycle
var CoolTime
var TriggerCondition = 0b00000001
var seg_num

# excitcodes
exitcode endLabel = 0xffffffffffffffff

set_shutter(ShutterInitialization)
update()

def init_pulse_shelving():
    set_shutter(INIT_ShelvingLaser)
    update(InitPulseTime)
    set_inv_shutter(INIT_ShelvingLaser)
    update()

def pulse_shelving():
    set_shutter(ShelvingLaser)
    update(PulseTime)
    set_inv_shutter(ShelvingLaser)
    update()

def Ramsey_pulse_shelving():
    set_shutter(ShelvingLaser)
    update(PulseTime)
    set_inv_shutter(ShelvingLaser)
    update()

def AWG_event_trigger():
    set_shutter(AWGTrigger)
    update(AWGEventTriggerWait)
    set_inv_shutter(AWGTrigger)
    update(AWGEventTriggerWait)

def repump_shelve():
    set_shutter(RepumpShelve)
    update(RepumpTime)
    set_inv_shutter(AllLasersOn)
    update()

def get_PMT_count():
    set_counter(PMT)
    update(PMTIntTime)
    clear_counter()
    update()

def turn_on_fluorescence():
    set_shutter(FluorescenceLasers)
    update()

def turn_off_fluorescence():
    set_inv_shutter(FluorescenceLasers)
    update()

def optical_pump():
    set_shutter(OpticalPumpLasers)
    update(OpticalPumpTime)
    set_inv_shutter(OpticalPumpLasers)
    update()

def F1_pump():
    set_shutter(F1PumpLasers)
    update(F1PumpTime)
    set_inv_shutter(AllLasersOn)
    update()
    set_shutter(Just650Laser)
    update(GeneralWaitTime)
    set_inv_shutter(Just650Laser)
    update()

def F1_pump_long():
    set_shutter(F1PumpLasers)
    update(F1PumpTime_long)
    set_inv_shutter(AllLasersOn)
    update()
    set_shutter(Just650Laser)
    update(GeneralWaitTime)
    set_inv_shutter(Just650Laser)
    update()

def shelving_cool_one():
    set_shutter(RepumpShelve)
    update(RepumpTimeForCool)
    set_inv_shutter(RepumpShelve)
    update()
    set_shutter(OpticalPumpLasers)
    update(OpticalPumpTimeForCool)
    set_inv_shutter(OpticalPumpLasers)
    update()
    set_shutter(ShelvingCoolLaserOne)
    update(CoolTime)
    set_inv_shutter(ShelvingLaser)
    update()

def shelving_cool_two():
    set_shutter(RepumpShelve)
    update(RepumpTimeForCool)
    set_inv_shutter(RepumpShelve)
    update()
    set_shutter(OpticalPumpLasers)
    update(OpticalPumpTimeForCool)
    set_inv_shutter(OpticalPumpLasers)
    update()
    set_shutter(ShelvingCoolLaserTwo)
    update(CoolTime)
    set_inv_shutter(ShelvingLaser)
    update()

while not pipe_empty():
    apply_next_scan_point()
    ExperimentCount = 0
    while ExperimentCount<Experiments:

        #set_shutter(Shutter614nm)
        #update(MechShutter614Wait)



        set_shutter(RepumpShelve)
        update()

        if CoolTimeFluor>0:
            turn_on_fluorescence()
            update(CoolTimeFluor)
            turn_off_fluorescence()

        set_inv_shutter(AllLasersOn)
        update()
        CoolCycle = 0
        CoolTime = CoolTimeStart
        while CoolCycle<CoolCycleNumOne:
            CoolTime = CoolTime + CoolPulseStep
            shelving_cool_one()
            #shelving_cool_two()
            CoolCycle = CoolCycle + 1
        
        if CoolCycleNumOne > 0:
            AWG_event_trigger()
            AWG_event_trigger()


        repump_shelve()

        seg_num = 1
        F1Repetitions = 0
        while F1Repetitions < F1PumpReps:

            update(GeneralWaitTime)
            if F1PumpTime>0:
                F1_pump()
            update(GeneralWaitTime)

            init_pulse_shelving()
            update(GeneralWaitTime)
            repump_shelve()
            update(GeneralWaitTime)
            F1Repetitions = F1Repetitions + 1
            update(GeneralWaitTime)


        Repetitions = 0
        while Repetitions<InitReps:
            repump_shelve()
            update(GeneralWaitTime)
            init_pulse_shelving()
            update(GeneralWaitTime)
            Repetitions = Repetitions + 1

        if RepumpTime>0:
            repump_shelve()

        AWG_event_trigger()
        seg_num = seg_num + 1
        update(AWGEventTriggerWait)

        pulse_shelving()
        seg_num = seg_num + 2 #Shelving for herald 
        update()

        turn_on_fluorescence()
        get_PMT_count()
        turn_off_fluorescence() 

        #pulse_shelving()
        #seg_num = seg_num + 2 #de-shelving for herald
        update()

        wait_trigger(TriggerCondition)

        Ramsey_pulse_shelving()

        seg_num = seg_num + 2

        turn_on_fluorescence()
        get_PMT_count()
        turn_off_fluorescence()
        
        update()
        while seg_num < Total_segs:

            pulse_shelving()


            turn_on_fluorescence()
            get_PMT_count()
            turn_off_fluorescence()


            update()
            seg_num = seg_num + 2

        AWG_event_trigger()
        ExperimentCount = ExperimentCount + 1
    

exit( endLabel )

In [46]:
int(np.log2(4))

2

In [44]:
x = np.array([[2,2,2,2],[1,2,3,4]])
threshold = 2
mean_value = np.mean(np.array(x)>threshold,axis = 0)
mean_value

array([0. , 0. , 0.5, 0.5])

In [21]:
def Z_gate_states(sub_dim, hidden_string):

    n = int(np.log2(sub_dim))
    if len(hidden_string) != n:
        raise ValueError("Hidden string length must equal log2(sub_dim).")
    s_bits = [int(bit) for bit in hidden_string]
    list_of_bits = []
    for x in range(sub_dim):
        x_bits = [int(b) for b in format(x, f'0{n}b')]
        dot = sum(s * xb for s, xb in zip(s_bits, x_bits)) % 2
        if dot == 1:
            list_of_bits.append(x+1)
    return list_of_bits

bv_oracle(4, '01')

[2, 4]

In [55]:
import numpy as np
#=====================HELPER_FUNCTIONS=============================
def fix_couplings_and_phases(couplings, fixed_phase_flags, Virtual_Z=False, Z_gate_states=None):
    new_couplings = []
    new_fixed_phase_flags = []
    for (cpl, phase_flag) in zip(couplings, fixed_phase_flags):
        i, j = cpl
        if i != 0 and j == 0:
            cpl_fixed = (0, i)
            phase_flag_fixed = phase_flag + 1.0
        else:
            cpl_fixed = cpl
            phase_flag_fixed = phase_flag

        if Virtual_Z and Z_gate_states is not None:
            if i in Z_gate_states or j in Z_gate_states:
                phase_flag_fixed += 1.0
        new_couplings.append(cpl_fixed)
        new_fixed_phase_flags.append(phase_flag_fixed)
    return new_couplings, new_fixed_phase_flags


def parse_string_to_int_list(s):
    s = s.strip()
    if s.startswith('[') and s.endswith(']'):
        inside = s[1:-1].strip()
        parts = inside.split(',')
        return [int(x) for x in parts]
    else:
        return [int(s)]

def build_transitions_list(index_to_str_map, couplings):
    transitions_list = []
    for (start_idx, end_idx) in couplings:
        left_list  = parse_string_to_int_list(index_to_str_map[start_idx])
        right_list = parse_string_to_int_list(index_to_str_map[end_idx])
        transitions_list.append(left_list + right_list)
    return transitions_list

def compute_pi_times(pi_t):
    transition_strengths = np.loadtxt(
        'Z:\\Lab Data\\Phase_and_freq_correction_180Hz\\Transition_strengths_4p216.txt', delimiter=','
    )
    transition_strengths[transition_strengths == 0] = np.nan

    # pi_t = np.array([19.470, 35.554, 41.166, 30.108, 39.326])
    strengths = np.array([
        transition_strengths[23, 0], transition_strengths[14, 0],
        transition_strengths[17, 4], transition_strengths[16, 4], transition_strengths[15, 4]
    ])

    factors = np.array(pi_t) * strengths
    Fs = [1, 2, 3, 4]
    row_labels = [[i, i - j] for i in Fs for j in range(2 * i + 1)]
    col_labels = [-2, -1, 0, 1, 2]

    pi_times = np.zeros((24, 5))
    for i in range(np.shape(transition_strengths)[0]):
        for j in range(np.shape(transition_strengths)[1]):
            if not np.isnan(transition_strengths[i, j]):
                delta_m = (row_labels[i][1] - col_labels[j]) + 2
                pi_times[i, j] = factors[delta_m] / transition_strengths[i, j]
    
    return pi_times

def get_pi_times(transitions,matrix):
    pi_times_list = []
    for transition in transitions:
        row_label = [transition[1],transition[2]]
        Fs = [1,2,3,4]
        states = []
        for i in Fs:
            for j in range(2*i+1):
                mF = i-j
                states.append([i,mF])
    
        row_labels = states
        col_labels = [-2,-1, 0, 1, 2]
        col_label = transition[0]
    
        row_index = next((i for i, label in enumerate(row_labels) if label == row_label), None)

        col_index = col_labels.index(col_label)
        
        if row_index is not None and col_index in range(len(col_labels)):
            pi_times_list.append(matrix[row_index, col_index])
        else:
            pi_times_list.append(np.nan)

    return pi_times_list

def pulse_duration_for_fraction(f, pi_times):
    Omega = np.pi/np.array(pi_times)
    # if not (0 <= f <= 1):
    #     raise ValueError("Fraction f must be between 0 and 1.")
    theta = np.pi*f # 2.0 * np.arcsin(np.sqrt(f))
    return theta / Omega 

#=====================GATE_FUNCTIONS=============================

def Bussed_PolyQubit_Hadamard(dim, mappings,pi_t = [23.172,40.114,47.954,34.418,46.591], U1 = False, Virtual_Z = False, Z_gate_state = None):
    if dim == 2:
        # couplings = [(2,0),(0,1),(2,0)] + [(4,0),(3,0)]  + [(0,1),(3,0)] + [(0,2),(4,0)] 
        # theta = [1,1/2,1] + [1,1/2]  + [1/2,1] + [1/2,1]      
        # fixed_phase_flags = [0.5,0.5,0.5] + [0.5,0.5]  + [0.5,0.5] + [0.5,0.5]
        # couplings = [(0, 2), (3, 0), (0, 1), (4, 0), (0, 1), (3, 0), (2, 0)]
        # theta = [1, 0.5, 2.0 * np.arcsin(np.sqrt(1/3))/np.pi, 2/3, 2.0 * np.arcsin(np.sqrt(1/3))/np.pi, 0.5, 1]
        # fixed_phase_flags = [1,0,0,0,0,0,0,0]
        # couplings, fixed_phase_flags = fix_couplings_and_phases(couplings, fixed_phase_flags,Virtual_Z = False, Z_gate_states = None)
        # probe_couplings = [(0,1),(0,2),(0,3),(0,4)]

        couplings = [(0, 3), (0, 1), (0, 2), (0, 1), (0, 3)]
        theta = [0.5, 2.0 * np.arcsin(np.sqrt(1/3))/np.pi, 4/3, 2.0 * np.arcsin(np.sqrt(2/3))/np.pi + 1, 0.5]
        fixed_phase_flags = [1.5, 0.5, 1.5, 0.5, 0.5]
        couplings, fixed_phase_flags = fix_couplings_and_phases(couplings, fixed_phase_flags,Virtual_Z = Virtual_Z, Z_gate_states = Z_gate_state)
        probe_couplings = [(0,1),(0,2),(0,3)]
        
    if dim == 3:
        if U1:
            couplings = [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7)]
            theta = [2*np.arcsin(np.sqrt(1/8))/np.pi, 2*np.arcsin(np.sqrt(1/7))/np.pi, 2*np.arcsin(np.sqrt(1/6))/np.pi, 2*np.arcsin(np.sqrt(1/5))/np.pi, 2*np.arcsin(np.sqrt(1/4))/np.pi, 2*np.arcsin(np.sqrt(1/3))/np.pi, 2*np.arcsin(np.sqrt(1/2))/np.pi+1]
            fixed_phase_flags = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
            
        else:
            couplings =  [(0, 6), (0, 1), (0, 3), (0, 5), (0, 4), (0, 2), (0, 4), (0, 5), (0, 7), (0, 3), (0, 1), (0, 2), (0, 4), (0, 7), (0, 1), (0, 5), (0, 3), (0, 6), (0, 7), (0, 1), (0, 6)]
            theta =  [1.5, 1, 1., 1.5, 2.0 * np.arcsin(np.sqrt(1/3))/np.pi, 2/3, 1 + 2.0 * np.arcsin(np.sqrt(1/3))/np.pi, 1, 1, 0.5, 0.5, 0.5, 1.0, 1.5, 1.0, 1.5, 1, 1.5, 1.5, 1, 1.5]
            fixed_phase_flags =  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1.5, 1.5, 0.5, 0.5, 0.5, 1.5, 0.5, 0.5, 1.5, 1.5, 0.5, 1.5, 1.5]

        couplings, fixed_phase_flags = fix_couplings_and_phases(couplings, fixed_phase_flags,Virtual_Z = False, Z_gate_states = None)
        probe_couplings = [(0,1),(0,2),(0,3),(0,4),(0,5),(0,6),(0,7)]
    Hadamard_trans = build_transitions_list(mappings,couplings)
    Hadamard_pitimes = list(pulse_duration_for_fraction(np.array(theta),get_pi_times(Hadamard_trans,compute_pi_times(pi_t))))
    probe_trans = build_transitions_list(mappings,probe_couplings)
    return Hadamard_trans, Hadamard_pitimes,fixed_phase_flags, theta, probe_trans



mappings = {0: '1', 1: '[2, -1]', 2: '[3, 2]', 3: '[4, 2]', 4: '[4, 0]', 5: '[3, 3]', 6: '[4, 1]', 7: '[4, -1]'}
# mappings = {0: '-2', 1: '[3, -3]', 2: '[3, -2]', 3: '[4, -4]'}
print(Bussed_PolyQubit_Hadamard(3,mappings))

Hadamard_trans, Hadamard_pitimes,fixed_phase_flags, theta, probe_trans = Bussed_PolyQubit_Hadamard(3,mappings, U1 = True)
print(sum(Hadamard_pitimes))

([[1, 4, 1], [1, 2, -1], [1, 4, 2], [1, 3, 3], [1, 4, 0], [1, 3, 2], [1, 4, 0], [1, 3, 3], [1, 4, -1], [1, 4, 2], [1, 2, -1], [1, 3, 2], [1, 4, 0], [1, 4, -1], [1, 2, -1], [1, 3, 3], [1, 4, 2], [1, 4, 1], [1, 4, -1], [1, 2, -1], [1, 4, 1]], [73.89308500550862, 70.99233027023448, 55.44758693595854, 81.42355354079477, 20.28519942180788, 39.69332425943624, 72.05606414927871, 54.282369027196516, 44.83222775651393, 27.72379346797927, 35.49616513511724, 29.769993194577182, 51.77086472747083, 67.2483416347709, 70.99233027023448, 81.42355354079477, 55.44758693595854, 73.89308500550862, 67.2483416347709, 70.99233027023448, 73.89308500550862], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1.5, 1.5, 0.5, 0.5, 0.5, 1.5, 0.5, 0.5, 1.5, 1.5, 0.5, 1.5, 1.5], [1.5, 1, 1.0, 1.5, 0.3918265520306073, 0.6666666666666666, 1.3918265520306072, 1, 1, 0.5, 0.5, 0.5, 1.0, 1.5, 1.0, 1.5, 1, 1.5, 1.5, 1, 1.5], [[1, 2, -1], [1, 3, 2], [1, 4, 2], [1, 4, 0], [1, 3, 3], [1, 4, 1], [1, 4, -1]])
165.79378831413615


In [73]:
def fix_couplings_and_phases(couplings, fixed_phase_flags, Virtual_Z=False, Z_gate_states=None):
    new_couplings = []
    new_fixed_phase_flags = []
    for (cpl, phase_flag) in zip(couplings, fixed_phase_flags):
        i, j = cpl
        if i != 0 and j == 0:
            cpl_fixed = (0, i)
            phase_flag_fixed = phase_flag + 1.0
        else:
            cpl_fixed = cpl
            phase_flag_fixed = phase_flag

        if Virtual_Z and Z_gate_states is not None:
            if i in Z_gate_states or j in Z_gate_states:
                phase_flag_fixed += 1.0
        new_couplings.append(cpl_fixed)
        new_fixed_phase_flags.append(phase_flag_fixed)
    return new_couplings, new_fixed_phase_flags

couplings = [(0, 2), (3, 0), (0, 1), (4, 0), (0, 1), (3, 0), (2, 0)]
fractions = [1, 0.5, 2.0 * np.arcsin(np.sqrt(1/3))/np.pi, 2/3, 2.0 * np.arcsin(np.sqrt(1/3))/np.pi, 0.5, 1]
fixed_phase_flags = [1,0,0,0,0,0,0,0]
couplings, fixed_phase_flags = fix_couplings_and_phases(couplings, fixed_phase_flags,Virtual_Z = True, Z_gate_states = [1,2,3,4])
print(couplings, fixed_phase_flags)

[(0, 2), (0, 3), (0, 1), (0, 4), (0, 1), (0, 3), (0, 2)] [2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 2.0]


In [87]:
x = np.array([np.nan,np.nan,np.nan])
if np.isnan(x.all()):
    print('true')

In [91]:
def Z_gate_states(sub_dim, hidden_string):

    n = int(np.log2(sub_dim))
    if len(hidden_string) != n:
        raise ValueError("Hidden string length must equal log2(sub_dim).")
    s_bits = [int(bit) for bit in hidden_string]
    list_of_bits = []
    for x in range(sub_dim):
        x_bits = [int(b) for b in format(x, f'0{n}b')]
        dot = sum(s * xb for s, xb in zip(s_bits, x_bits)) % 2
        if dot == 1:
            list_of_bits.append(x+1)
    return list_of_bits
print('Z_gate_states',Z_gate_states(4, "01"))

Z_gate_states [2, 4]


In [20]:
def Z_gate_states(sub_dim, hidden_string):

    n = int(np.log2(sub_dim))
    if len(hidden_string) != n:
        raise ValueError("Hidden string length must equal log2(sub_dim).")
    s_bits = [int(bit) for bit in hidden_string]
    list_of_bits = []
    for x in range(sub_dim):
        x_bits = [int(b) for b in format(x, f'0{n}b')]
        dot = sum(s * xb for s, xb in zip(s_bits, x_bits)) % 2
        if dot == 1:
            list_of_bits.append(x)
    return list_of_bits

Z_gate_states(4,'00')

[]

In [31]:

def Bussed_PolyQubit_Hadamard(dim, mappings,pi_t = [23.172,40.114,47.954,34.418,46.591],Virtual_Z = False, Z_gate_state = None):
    if dim == 2:
        # couplings = [(2,0),(0,1),(2,0)] + [(4,0),(3,0)]  + [(0,1),(3,0)] + [(0,2),(4,0)] 
        # theta = [1,1/2,1] + [1,1/2]  + [1/2,1] + [1/2,1]      
        # fixed_phase_flags = [0.5,0.5,0.5] + [0.5,0.5]  + [0.5,0.5] + [0.5,0.5]
        #couplings = [(0, 2), (3, 0), (0, 1), (4, 0), (0, 1), (3, 0), (2, 0)]
        #theta = [1, 0.5, 2.0 * np.arcsin(np.sqrt(1/3))/np.pi, 2/3, 2.0 * np.arcsin(np.sqrt(1/3))/np.pi, 0.5, 1]
        #fixed_phase_flags = [1,0,0,0,0,0,0,0]
        
        couplings = [(0, 3), (0, 1), (0, 2), (0, 1), (0, 3)]
        theta = [0.5, 2.0 * np.arcsin(np.sqrt(1/3))/np.pi, 4/3, 2.0 * np.arcsin(np.sqrt(2/3))/np.pi + 1, 0.5]
        fixed_phase_flags = [1.5, 0.5, 1.5, 0.5, 0.5]
        
        couplings, fixed_phase_flags = fix_couplings_and_phases(couplings, fixed_phase_flags,Virtual_Z = Virtual_Z, Z_gate_states = Z_gate_state)
        probe_couplings = [(0,1),(0,2),(0,3)]
    if dim == 3:
    
    
        couplings =  [(0, 6), (0, 1), (0, 3), (0, 5), (0, 4), (0, 2), (0, 4), (0, 5), (0, 7), (0, 3), (0, 1), (0, 2), (0, 4), (0, 7), (0, 1), (0, 5), (0, 3), (0, 6), (0, 7), (0, 1), (0, 6)]
        theta =  [1.5, 1, 1., 1.5, 2.0 * np.arcsin(np.sqrt(1/3))/np.pi, 2/3, 1 + 2.0 * np.arcsin(np.sqrt(1/3))/np.pi, 1, 1, 0.5, 0.5, 0.5, 1.0, 1.5, 1., 1.5, 1, 1.5, 1.5, 1, 1.5]
        fixed_phase_flags =  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1.5, 1.5, 0.5, 0.5, 0.5, 1.5, 0.5, 0.5, 1.5, 1.5, 0.5, 1.5, 1.5]

    
        #couplings = [(2,0),(0,1),(2,0)] + [(4,0),(3,0)]  + [(0,1),(3,0)] + [(0,2),(4,0)]\
        #          + [(6,0),(0,5),(6,0)] + [(8,0),(7,0)]  + [(0,5),(7,0)] + [(6,0)]\
        #          + [(0,2),(6,0)] + [(5,0),(0,1),(5,0)] + [(7,0),(0,3),(7,0)] + [(0,4),(8,0)]
        #theta = [1,1/2,1] + [1,1/2]  + [1/2,1] + [1/2,1]\
        #          + [1,1/2,1] + [1,1/2]  + [1/2,1] + [1/2]\
        #          + [1/2,1] + [1,1/2,1] + [1,1/2,1] + [1/2,1]
        #fixed_phase_flags = [0.5,0.5,0.5] + [0.5,0.5]  + [0.5,0.5] + [0.5,0.5]\
        #                   +[0.5,0.5,0.5] + [0.5,0.5]  + [0.5,0.5] + [0.5]\
        #                   + [0.5,0.5] + [0.5,0.5,0.5] + [0.5,0.5,0.5] + [0.5,0.5]
        couplings, fixed_phase_flags = fix_couplings_and_phases(couplings, fixed_phase_flags,Virtual_Z = Virtual_Z, Z_gate_states = Z_gate_state)
        probe_couplings = [(0,1),(0,2),(0,3),(0,4),(0,5),(0,6),(0,7)]
    Hadamard_trans = build_transitions_list(mappings,couplings)
    Hadamard_pitimes = list(pulse_duration_for_fraction(np.array(theta),get_pi_times(Hadamard_trans,compute_pi_times(pi_t))))
    probe_trans = build_transitions_list(mappings,probe_couplings)
    return Hadamard_trans, Hadamard_pitimes,fixed_phase_flags, theta, probe_trans

mappings = {0: '-2', 1: '[3, -3]', 2: '[3, -2]', 3: '[4, -4]'}
dim = 2
Bussed_PolyQubit_Hadamard(dim, mappings)

([[-2, 4, -4], [-2, 3, -3], [-2, 3, -2], [-2, 3, -3], [-2, 4, -4]],
 [11.586, 15.717730308155781, 66.76839543716886, 64.51026969184421, 11.586],
 [1.5, 0.5, 1.5, 0.5, 0.5],
 [0.5, 0.3918265520306073, 1.3333333333333333, 1.6081734479693928, 0.5],
 [[-2, 3, -3], [-2, 3, -2], [-2, 4, -4]])

In [34]:
2/3

0.6666666666666666

In [36]:
2.0 * np.arcsin(np.sqrt(1/3))/np.pi

0.3918265520306073

In [2]:
sum([14.112748941791473, 13.436561432710828, 13.570872939568773, 13.328738834766018, 16.56870630361711, 16.907327343424164, 58.122536091216304, 64.72504449661488, 61.34552019864338, 50.690456346401696, 74.55917836627697, 17.693541680598216, 36.30251421977515, 62.850107995216156, 49.706118910851316, 38.7483573941442, 25.345228173200848, 30.67276009932169, 27.226885664831364, 45.15656631461794, 58.122536091216304, 61.34552019864338, 74.55917836627697, 50.690456346401696, 64.72504449661488, 58.122536091216304, 61.34552019864338, 64.72504449661488])

1224.705608033216

In [4]:
sum([14.112748941791473, 13.436561432710828, 13.570872939568773, 13.328738834766018, 16.56870630361711, 16.907327343424164, 58.122536091216304])

146.04749188709468